In [7]:
# 設定工作目錄
import os
os.chdir('C:/python-training/eyeglad')
print(os.getcwd())
# 添加模組路徑
import sys
sys.path.append('C:/python-training/eyeglad/use_SQL')

import dash
from dash import dash_table, dcc, html, Input, Output, State
import dash_bootstrap_components as dbc
import pandas as pd
import plotly.express as px
from use_SQL import getdata, log_in
from sqlalchemy import inspect


C:\python-training\eyeglad


In [8]:
database = 'amazon'

# 連接到 MySQL 資料庫
engine = log_in.log_in_mySQL(database)

# 建立檢查器
inspector = inspect(engine)

# 查詢所有表格
tables = inspector.get_table_names()
print("Tables:", '\n'.join(tables))


Log in successful
Tables: sponsored_brands_attributed_purchases_report
sponsored_brands_campaign_placement_report
sponsored_brands_campaign_report
sponsored_brands_category_benchmark_report
sponsored_brands_gross_and_invalid_traffic_report
sponsored_brands_keyword_placement_report
sponsored_brands_keyword_report
sponsored_brands_search_term_impression_share_report
sponsored_brands_search_term_report
sponsored_products_advertised_product_report
sponsored_products_budget_report
sponsored_products_campaign_report
sponsored_products_gross_and_invalid_traffic_report
sponsored_products_performance_over_time_report
sponsored_products_placement_report
sponsored_products_purchased_product_report
sponsored_products_search_term_impression_share_report
sponsored_products_search_term_report
sponsored_products_targeting_report


In [9]:

# 初始化Dash應用
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

# 應用佈局
app.layout = dbc.Container([
    dbc.Row(dbc.Col(html.H1("電商平台分析儀表板"), className="mb-4")),
    
    dbc.Row([
        dbc.Col([
            html.Label("選擇表格"),
            dcc.Dropdown(
                id='table-dropdown',
                options=[{'label': table, 'value': table} for table in tables],
                value=tables[0],  # 默認選擇第一個表格
                className="mb-2"
            ),
        ], width=4)
    ]),
    
    dbc.Row([
        dbc.Col([
            html.Label("選擇X軸"),
            dcc.Dropdown(
                id='x-axis',
                className="mb-2"
            ),
            html.Label("選擇Y軸"),
            dcc.Dropdown(
                id='y-axis',
                className="mb-2"
            ),
            html.Label("篩選條件"),
            dcc.Dropdown(
                id='filter-column',
                className="mb-2"
            ),
            dcc.Input(
                id='filter-value',
                placeholder='輸入篩選值...',
                type='text',
                className="mb-2"
            )
        ], width=4),
        
        dbc.Col(dcc.Graph(id='scatter-plot'), width=8)
    ]),
    
    dbc.Row(dbc.Col(html.H2("數據表"), className="mt-4")),
    
    dbc.Row(dbc.Col(dash_table.DataTable(
        id='table',
        page_size=10
    )))
], fluid=True)

# 回調函數動態加載表格數據
@app.callback(
    [Output('x-axis', 'options'),
     Output('x-axis', 'value'),
     Output('y-axis', 'options'),
     Output('y-axis', 'value'),
     Output('filter-column', 'options'),
     Output('filter-column', 'value'),
     Output('table', 'columns'),
     Output('table', 'data')],
    [Input('table-dropdown', 'value')]
)
def update_columns(table_name):
    df = getdata.fetch_data_from_db(database=database, table=table_name)
    options = [{'label': col, 'value': col} for col in df.columns]
    columns = [{"name": i, "id": i} for i in df.columns]
    data = df.to_dict('records')
    return options, df.columns[0], options, df.columns[1], options, df.columns[2], columns, data

# 回調函數更新圖表和表格
@app.callback(
    [Output('scatter-plot', 'figure'),
     Output('table', 'data')],
    [Input('x-axis', 'value'),
     Input('y-axis', 'value'),
     Input('filter-column', 'value'),
     Input('filter-value', 'value'),
     State('table-dropdown', 'value')]
)
def update_chart(x_axis, y_axis, filter_column, filter_value, table_name):
    df = getdata.fetch_data_from_db(database=database, table=table_name)
    # 過濾數據
    if filter_value:
        filtered_df = df[df[filter_column].astype(str).str.contains(filter_value)]
    else:
        filtered_df = df

    # 創建圖表
    fig = px.scatter(filtered_df, x=x_axis, y=y_axis, color=filter_column)
    
    # 更新表格數據
    table_data = filtered_df.to_dict('records')
    
    return fig, table_data

# 運行應用
if __name__ == '__main__':
    app.run_server(debug=True)


OSError: Address 'http://127.0.0.1:8050' already in use.
    Try passing a different port to run_server.